In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.vision.all import *

In [ ]:
#Unzip image files
from zipfile import ZipFile
with ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip', 'r') as archive:
    archive.extractall('/kaggle/working/')

In [ ]:
#Set path to working folder
path = Path('/kaggle/working/')
path.ls()

In [ ]:
#Set the pets dataloader
pets = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=RandomSplitter(seed=42),
                 get_y=using_attr(RegexLabeller(r'(.+)\.\d+.jpg$'), 'name'),
                 item_tfms=Resize(460),
                 batch_tfms=aug_transforms(size=224, min_scale=0.75))
dls = pets.dataloaders(path/'train')

In [ ]:
#Train a very simple model
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(2)

In [ ]:
#Unzip image files for testing

with ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip', 'r') as archive:
    archive.extractall('/kaggle/working/')

In [ ]:
#data loader pointed to test dataset and get predictions
test_dl = dls.test_dl(get_image_files(path/'test'))
allPreds = learn.get_preds(dl=test_dl)

In [ ]:
#Match filename to prediction probs
allFiles = list((path/'test').ls())
guesses = [[int(re.findall(r'.+/(\d+).jpg$', str(allFiles[i]))[0]), float(allPreds[0][i][1])] for i in range(len(allFiles))]

In [ ]:
#Generate submission file...download this file and 
#manually upload it here: https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/submit
submissionFile = pd.DataFrame(guesses,columns=['id','label']).sort_values('id')
submissionFile.to_csv('submissionFile_01March.csv',index=False)